In [2]:
from langchain_groq.chat_models import ChatGroq
from dotenv import load_dotenv
_=load_dotenv()

# api_key = os.getenv('GROQ_API_KEY') 

llm = ChatGroq(model="openai/gpt-oss-20b")


/home/mahavir/code/python_projects/rag_project/backend/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
from langchain.document_loaders import  CSVLoader

In [4]:
CSV_FILE_PATH = "../data/faq.csv"
df = pd.read_csv(CSV_FILE_PATH,encoding='latin1')

In [5]:

csv_loader = CSVLoader(CSV_FILE_PATH,source_column='prompt',encoding='latin1')
raw_data = csv_loader.load()
raw_data

[Document(metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}, page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.'),
 Document(metadata={'source': 'Why should I trust Codebasics?', 'row': 1}, page_content='prompt: Why should I trust Codebasics?\nresponse: Till now 9000 + learners have benefitted from the quality of our courses. You can check the review section and also we have attached their LinkedIn profiles so that you can connect with them and ask directly.'),
 Document(metadata={'source': 'Is there any prerequisite for taking this bootcamp ?', 'row': 2}, page_content='prompt: Is there any prerequisite for taking this bootcamp ?\nresponse: Our bootcamp is specifically designed for

In [6]:
raw_data[0].page_content

'prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.'

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
embeddings = HuggingFaceEmbeddings()
embeddings

/tmp/ipykernel_56950/750672620.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_56950/750672620.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
response = embeddings.embed_query("What is your refund policy")
response

[0.019298570230603218,
 0.035125911235809326,
 0.004028317518532276,
 -0.004395237658172846,
 -0.04093785956501961,
 -0.011212360113859177,
 -0.06468949466943741,
 0.004872607067227364,
 -0.030528485774993896,
 -0.03128640353679657,
 0.0588015541434288,
 -0.044457290321588516,
 0.05164077505469322,
 0.05633554235100746,
 0.016944054514169693,
 0.045635759830474854,
 -0.03485608845949173,
 -0.017647314816713333,
 0.07084163278341293,
 -0.052444618195295334,
 0.021447496488690376,
 0.014228218235075474,
 -0.010771779343485832,
 0.0037751535419374704,
 -0.0366893894970417,
 0.0128193823620677,
 0.02942175790667534,
 0.024480149149894714,
 0.008213852532207966,
 0.0034048575907945633,
 0.07124356180429459,
 -0.04399518668651581,
 0.019464882090687752,
 0.008417245000600815,
 1.4973029465181753e-06,
 -0.037470921874046326,
 -0.012717313133180141,
 -0.03941580653190613,
 0.004655722063034773,
 0.07392476499080658,
 0.03329242393374443,
 0.015014010481536388,
 -0.008270286954939365,
 0.004808

In [10]:
len(response)

768

as you can see it has give response of 788 tokens which doesnt make any sense at all now we will give out csv data as retrival so that llm can retrive that info and can act like our customer assistant

In [11]:
from langchain.vectorstores import FAISS
vector_db = FAISS.from_documents(raw_data,embeddings)
vector_db


In [12]:
retreiver = vector_db.as_retriever()
retreiver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x739a2239a120>, search_kwargs={})

In [13]:
response = retreiver.invoke("What is your refund policy")
response

[Document(id='e28e0a0a-7a65-4645-bdf7-98e407e41bf9', metadata={'source': 'What if I don\x92t like this bootcamp?', 'row': 6}, page_content='prompt: What if I don\x92t like this bootcamp?\nresponse: As promised we will give you a 100% refund based on the guidelines (Please refer to our course refund policy before enrolling).'),
 Document(id='c0fb5aa3-8dfe-4b75-a4f2-b1f5d72e27ec', metadata={'source': 'Once purchased, is this course available for lifetime access?', 'row': 22}, page_content='prompt: Once purchased, is this course available for lifetime access?\nresponse: Yes'),
 Document(id='e4b9ce3c-0a6e-4472-9606-b4960c215742', metadata={'source': 'How can I get help if I have a doubt and need support?', 'row': 23}, page_content='prompt: How can I get help if I have a doubt and need support?\nresponse: We have an active discord server where you can post your question and most of the time you will get an answer in a reasonable time frame.'),
 Document(id='4eb1b193-6786-4428-8993-5feea46ba

In [14]:
response = retreiver.invoke("How can I rearrange the column order in the P&L check? I would like to change the order of the columns.")
response

[Document(id='8700e647-212e-4fdb-a9e3-c057a78aeff5', metadata={'source': 'How can I rearrange the column order in the P&L check? I would like to change the order of the columns.', 'row': 66}, page_content='prompt: How can I rearrange the column order in the P&L check? I would like to change the order of the columns.\nresponse: Check this reference:\nhttps://discord.com/channels/1090613684163850280/1112342820930453516/1112399292792062022'),
 Document(id='107dfac1-1df0-4f13-8325-c702026f8441', metadata={'source': 'The year column is missing in the P&L check. How can I resolve this issue and obtain the year column?', 'row': 64}, page_content='prompt: The year column is missing in the P&L check. How can I resolve this issue and obtain the year column?\nresponse: Check this reference:\n https://discord.com/channels/1090613684163850280/1111101322406658098/1111137901816848494'),
 Document(id='90cce850-e992-4115-82cb-c4b74a63d947', metadata={'source': 'There are missing values in the P&L check

In [16]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retreiver,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [17]:
response = chain('Do you provide job assistance and also do you provide job gurantee?')
response

/tmp/ipykernel_56950/2498657611.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain('Do you provide job assistance and also do you provide job gurantee?')


{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': '**Job assistance:**  \n> “Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.”\n\n**Job guarantee:**  \n> “We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.”\n\n> “The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gi

In [18]:
response['result']

'**Job assistance:**  \n> “Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.”\n\n**Job guarantee:**  \n> “We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.”\n\n> “The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest an